In [ ]:
import sys;
sys.path.insert(0, '..')

## Chapter 7 Code Snippets and Listings

### Periodic patterns in sound waves (section 7.1.1)

For example, for $N = 8$, we can define discrete samples of the signal using the following Python code:

In [ ]:
from math import sqrt, pi, cos
N = 8
frequency = 1.7
samples = [1/sqrt(8)*cos(2 * pi * frequency * (t / N)) for t in range(N)]

We can visualize the wave and the 8 values we computed above:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

x = np.linspace(0, N, 50)
wave = [1/sqrt(8)*cos(2 * pi * frequency * (t/N)) for t in x]
plt.plot(x, wave, label='signal', color='red')
plt.scatter(range(N), samples)
plt.show()

### Periodic patterns in quantum states (section 7.1.2)

We can create a quantum state with amplitudes that are discrete samples of this complex signal with a given `theta` (for example, $\frac{\pi}{3}$) using the following code:

In [ ]:
from util import cis
from math import sin

theta = pi/3
state = [sqrt(1/N) * cis(k*theta) for k in range(N)]

Listing 7.1 Function for creating an `n`-qubit geometric sequence state with angle `theta`

In [ ]:
def geom(n, theta):
    N = 2**n
    return [sqrt(1/N) * cis(k*theta) for k in range(N)]

Let's use it to create a 3-qubit quantum state that is a geometric sequence with $\theta = \frac{\pi}{3}$:

In [ ]:
state = geom(3, pi/3)

Let's look at the phase of each of the amplitudes:

In [ ]:
from math import atan2

for k in range(len(state)):
    print("phase of amplitude ", k, ":", round(atan2(state[k].imag, state[k].real), 5))

Let's create a 3-qubit quantum state which is a geometric sequence with $\theta = \frac{\pi}{6}$:

In [ ]:
state = geom(3, pi/6)

Listing 7.2 Circuit for encoding a geometric sequence state with `n` qubits for a given frequency `v`

In [ ]:
from sim_circuit import *

def geometric_sequence_circuit(n, v):
    theta = v*2*pi/2**n

    q = QuantumRegister(n)
    qc = QuantumCircuit(q)

    for j in range(n):
        qc.h(q[j])

    for j in range(n):
        qc.p(2 ** j * theta, q[j])

    return qc

Let's we use this function to encode our example samples (`v = 1.7`):

In [ ]:
n = 3
v = 1.7
qc = geometric_sequence_circuit(n, v)
state = qc.run()

In [ ]:
from util import print_state_table

print_state_table(state)

In this state, all the amplitudes match those of the example signal, $\frac{1}{\sqrt{8}}$, and the directions reflect the expected frequency of 1.7. We can check this using the code below:

In [ ]:
from util import all_close

theta = v*2*pi/2**n
assert all_close(state, [sqrt(1/2**n) * cis(k*theta) for k in range(2**n)])

### Roots of unity and their geometric sequences (section 7.1.3)

Let's check that $\omega_N^N = 1$ for $N=8$:

In [ ]:
N = 8
omega = cis(2*pi/N)
print(abs(omega**N))

In code, we can express the power sequence corresponding to $\omega_N$ as:

In [ ]:
omega = cis(2*pi/N)
sequence = [omega**k for k in range(N)]

We can also use the `cis` function to do the same:

In [ ]:
N = 8
sequence_cis = [cis(l*2*pi/N) for l in range(N)]

assert all_close(sequence, sequence_cis)

### Converting from phase to magnitude encoding with the Hadamard gate (section 7.2)

For example, let's represent the frequency $\frac{1}{3}$ in a single-qubit state.
To do this, we will prepare a single-qubit geometric sequence state with the angle $\theta = \frac{\pi}{3}$:

In [ ]:
state = geom(1, pi/3)

In [ ]:
print_state_table(state)

If we apply an additional Hadamard gate, the magnitudes of both amplitudes will change.
Let's look at the following implementation:

In [ ]:
q = QuantumRegister(1)
qc = QuantumCircuit(q)

theta = pi/3
qc.h(q[0])
qc.p(theta, q[0])
qc.h(q[0])

state = qc.run()

In [ ]:
print_state_table(state)

In this state, the encoded angle is translated in the magnitudes of the amplitudes. The amplitudes of the resulting state have magnitudes $\cos \frac{\theta}{2}$ and $\sin \frac{\theta}{2}$:

In [ ]:
from util import is_close

theta = pi/3
assert is_close(abs(state[0]), cos(theta/2))
assert is_close(abs(state[1]), sin(theta/2))

### Computing sequence similarity with inner products (section 7.3)

For example, say we bought 4 apples, 2 oranges, 2 peaches, and 3 bananas. Let's assume one apple costs $1.20, one orange costs $1.50, one peach costs $2.00, and one banana costs $0.70. We can express these quantities and prices as lists:

In [ ]:
quantities = [4, 2, 2, 3]
prices = [1.2, 1.5, 2, 0.7]

To calculate the total price for the list of items, we multiply the quantity of each item by its price and add the results together:

In [ ]:
print(sum([quantities[k] * prices[k] for k in range(len(quantities))]))

Listing 7.3 Compute the inner product of two state vectors

In [ ]:
def inner(v1, v2):
    assert(len(v1) == len(v2))
    return sum(z1*z2.conjugate() for z1, z2 in zip(v1, v2))

### The classical (discrete) Fourier transform (section 7.4.1)

We can represent each Fourier basis in Python code with a list:

In [ ]:
N = 4
omega = cis(2*pi/N)

F_0 = [omega**(0*k) for k in range(N)]
F_1 = [omega**(1*k) for k in range(N)]
F_2 = [omega**(2*k) for k in range(N)]
F_3 = [omega**(3*k) for k in range(N)]

If we inspect the first basis, we see that its components are equal to 1:

In [ ]:
print(F_0)

Alternatively, we can use the following code to find the Fourier bases:

In [ ]:
N = 4

F_0 = [cis(k*0*2*pi/N) for k in range(N)]
F_1 = [cis(k*1*2*pi/N) for k in range(N)]
F_2 = [cis(k*2*2*pi/N) for k in range(N)]
F_3 = [cis(k*3*2*pi/N) for k in range(N)]

Let's return to our earlier example of samples from a sinusoidal wave:

In [ ]:
N = 8
frequency = 1.7
samples = [1/sqrt(N)*cos(2 * pi * frequency * (i / N)) for i in range(N)]

To get the first item of the DFT of this signal, we need to compute the inner product between the discrete signal and the corresponding Fourier basis ($F_0$). We can use the `inner` function defined in the previous section:

In [ ]:
F_0 = [cis(k*0*2*pi/N) for k in range(N)]
similarity = inner(samples, F_0)

print(round(similarity.real, 5) + 1j*round(similarity.imag, 5))

We can compute the entire sequence with a list comprehension:

In [ ]:
dft = [inner(samples, [cis(k*l*2*pi/N) for k in range(N)]) for l in range(8)]

for x in dft:
    print(round(x.real, 5) + 1j*round(x.imag, 5))

Let's check that these results match what we get when we use `numpy`:

In [ ]:
f = np.fft.fft(samples)

for x in f:
    print(round(x.real, 5) + 1j*round(x.imag, 5))

### Introducing the QFT and IQFT (section 7.4.2)

Listing 7.4 Compute the Fourier basis for a given `N` and `l`

In [ ]:
def fourier_basis(N, l):
    return [1/sqrt(N) * cis(k*l*2*pi/N) for k in range(N)]

Listing 7.5 Function for simulating the IQFT on a list representing a quantum state

In [ ]:
def icft(state):
    N = len(state)
    s = [state[k] for k in range(N)]

    for i in range(N):
        state[i] = inner(s, fourier_basis(N, i))

Listing 5.6 Function for simulating the QFT on a list representing a quantum state

In [ ]:
def cft(state):
    N = len(state)
    s = [state[k] for k in range(N)]

    for i in range(N):
        state[i] = inner(s, fourier_basis(N, -i))

### Quantum circuits for the QFT and IQFT (section 7.5)

Listing 7.7 Quantum circuits for QFT and IQFT

In [ ]:
def qft(qc, targets, swap=True):
    for j in range(len(targets))[::-1]:
        qc.h(targets[j])
        for k in range(j)[::-1]:
            qc.cp(pi * 2.0 ** (k - j), targets[j], targets[k])

    if swap:
        qc.mswap(targets)


def iqft(qc, targets, swap=True):
    for j in range(len(targets))[::-1]:
        qc.h(targets[j])
        for k in range(j)[::-1]:
            qc.cp(-pi * 2 ** (k - j), targets[j], targets[k])

    if swap:
        qc.mswap(targets)

class QFT(QuantumCircuit):
    def __init__(self, m, reversed=False, swap=True):
        super().__init__(QuantumRegister(m))
        targets = range(m)
        if reversed:
            targets = targets[::-1]

        qft(self, targets, swap)


class IQFT(QuantumCircuit):
    def __init__(self, m, reversed=False, swap=True):
        super().__init__(QuantumRegister(m))
        targets = range(m)
        if reversed:
            targets = targets[::-1]

        iqft(self, targets, swap)

Listing 7.8 Methods to append QFT and IQFT to a circuit instance

**Note:** these will be added to the `QuantumCircuit` class

In [ ]:
def append_qft(self, reg, reversed=False, swap=True):
    self.append(QFT(len(reg), reversed, swap), reg)

def append_iqft(self, reg, reversed=False, swap=True):
    self.append(IQFT(len(reg), reversed, swap), reg)

**Efficient classical implementation**

The following is a much more efficient version of the classical implementation of the IQFT shown in the previous section.

In [ ]:
def classical_inverse_fourier(state, step, targets):
    n = len(targets)
    sq2 = sqrt(2)
    sq2i = 1/sqrt(2)
    for j in range(n)[::-1]:
        dist = 2**j
        rot = cis(-pi/dist)
        rots = [1 for _ in range(dist)]
        r = 1
        for m in range(dist):
            rots[m] = r
            r = r*rot

        for l in range(2**(n-j-1)):
            i = 0
            for k in range(2*l*dist, (2*l+1)*dist):
                state[k] = sq2i*(state[k] + state[k+dist])
                state[k+dist] = (state[k] - sq2*state[k+dist])*rots[i]
                i += 1

### Understanding the IQFT step-by-step (section 7.5.1)

Here is how the successive division method works. We start by dividing a given decimal number (13) by 2 and record the quotient and remainder. Since $13 = 6\cdot2 + 1$, the quotient is 6 and the remainder is 1. The remainder (1) is the first binary digit of the given number (starting from the right).

Python has a built-in function called `divmod` that gives the quotient and remainder of a division. If we use it, we get the expected results:

In [ ]:
divmod(13, 2)

We continue the process by applying `divmod` to the quotients:

In [ ]:
divmod(6, 2)

In [ ]:
divmod(3, 2)

In [ ]:
divmod(1, 2)